In [1]:
from keras.preprocessing.image import load_img
image = load_img('./Flicker8k_Dataset/990890291_afc72be141.jpg', target_size=(224,224))

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import img_to_array
image = img_to_array(image)

In [3]:
from keras.applications.vgg16 import preprocess_input

In [4]:
image = image.reshape((1,image.shape[0],image.shape[1], image.shape[2]))
image = preprocess_input(image)

In [5]:
from os import listdir
from os import path
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input

In [8]:
def load_photos(directory):
    images = dict()
    for name in listdir(directory)[:40]:
        filename = path.join(directory,name)
        image = load_img(filename, target_size = (224,224))
        image  = img_to_array(image)
        image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
        image = preprocess_input(image)
        image_id = name.split('.')[0]
        images[image_id]=image
    return images

In [9]:
directory = './arvind/'
images = load_photos(directory)
print('Loaded Images: %d' % len(images))

Loaded Images: 10


In [10]:
from keras.layers import Input
from keras.applications.vgg16 import VGG16
#load the model
in_layer = Input(shape=(224,224,3))
model = VGG16(include_top=False, input_tensor=in_layer, pooling='avg')
model.summary()

W0709 15:21:35.602441 140450706577216 module_wrapper.py:136] From /home/arvind/.local/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0709 15:21:35.636606 140450706577216 module_wrapper.py:136] From /home/arvind/.local/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [11]:
from os import listdir
from os import path
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input

In [12]:
def extract_features(directory):
    in_layer = Input(shape = (224,224,3))
    model = VGG16(include_top=False, input_tensor=in_layer)
    model.summary()
    features = dict()
    for name in listdir(directory)[:40]:
        filename = path.join(directory,name)
        image = load_img(filename, target_size=(224,224))
        image = img_to_array(image)
        image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
        image = preprocess_input(image)
        feature = model.predict(image, verbose=0)
        image_id = name.split('.')[0]
        features[image_id] = feature
        print('>%s' %name)
    return features

In [13]:
directory = 'arvind'

In [14]:
features = extract_features(directory)
print('Extracted Features: %d' %len(features))
dump(features, open('features.pkl','wb'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [15]:
def load_doc(filename):
    file = open(filename,'r')
    text = file.read()
    file.close()
    return text

In [16]:
def load_descriptions(doc):
    mapping = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        if len(line)<2:
            continue
        image_id, image_desc = tokens[0], tokens[1:]
        image_id = image_id.split('.')[0]
        image_desc = ' '.join(image_desc)
        if image_id not in mapping:
            mapping[image_id] = image_desc
    return mapping

In [17]:
filename = 'Flickr8k_text/arvind_doc.txt'
doc = load_doc(filename)
descriptions  = load_descriptions(doc)
print('Loaded: %d' %len(descriptions))

Loaded: 10


In [18]:
def clean_descriptions(descriptions):
    re_punc = re.compile('[%s]' %re.escape(string.punctuation))
    for key, desc in descriptions.items():
        desc = desc.split()
        desc = [word.lower() for word in desc]
        desc = [re_punc.sub('',w) for w in desc]
        desc = [word for word in desc if len(word)>1]
        descriptions[key] =' '.join(desc)

In [19]:
def save_doc(descriptions, filename):
    lines = list()
    for key, desc in descriptions.items():
        lines.append(key + ' ' +desc)
        data = '\n'.join(lines)
        file = open(filename,'w')
        file.write(data)
        file.close()

In [20]:
import string
import re

In [21]:
filename = 'Flickr8k_text/arvind_doc.txt'
doc = load_doc(filename)
descriptions = load_descriptions(doc)
print('Loaded: %d' %len(descriptions))
clean_descriptions(descriptions)
all_tokens = ' '.join(descriptions.values()).split()
vocabulary = set(all_tokens)
print('Vocabulary Size: %d' %len(vocabulary))
save_doc(descriptions, 'descriptions.txt')

Loaded: 10
Vocabulary Size: 72


In [22]:
def load_clean_descriptions(filename):
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        image_id, image_desc = tokens[0], tokens[1:]
        descriptions[image_id] = ' '.join(image_desc)
    return descriptions  
    

In [23]:
descriptions = load_clean_descriptions('descriptions.txt')
print('Loaded %d' %len(descriptions))

Loaded 10


In [24]:
cnt = 0
descriptions_1 = {}
for key, value in descriptions.items():
    cnt+=1
    descriptions_1[key] = value
    if cnt == 40:
        break

In [25]:
len(descriptions_1)

10

In [26]:
desc_text = list(descriptions_1.values())

In [27]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(desc_text)
vocab_size = len(tokenizer.word_index)+1
print('Vocabulary Size: %d' %vocab_size)

Vocabulary Size: 73


In [28]:
sequences = tokenizer.texts_to_sequences(desc_text)

In [29]:
from keras.preprocessing.sequence import pad_sequences
max_length = max(len(s) for s in sequences)
print('Description Length: %d' %max_length)
# padded = pad_sequences(sequences, maxlen=max_length, padding='post')

Description Length: 16


In [30]:
from keras.utils import to_categorical
# y = to_categorical(padded, num_classes=vocab_size)

In [31]:
# y = y.reshape((len(descriptions_1),max_length,vocab_size))

In [32]:
# print(y.shape)

In [33]:
# in_seq, out_seq = seq[:i], seq[i]
# in_seq = pad_sequences([in_seq], maxlen = max_length)[0]
# out_seq = to_categorical([out_seq],num_classes=vocab_size)[0]

In [34]:
from numpy import array
X, y = list(), list()
for img_no, seq in enumerate(sequences):
    for i in range(1,len(seq)):
        in_seq, out_seq = seq[:i], seq[i]
        in_seq = pad_sequences([in_seq],maxlen=max_length)[0]
        out_seq = to_categorical([out_seq],num_classes=vocab_size)[0]
        X.append(in_seq)
        y.append(out_seq)

X,y = array(X), array(y)
print(X.shape)
print(y.shape)

(89, 16)
(89, 73)


In [35]:
def data_generator(descriptions, tokenizer, max_length):
    directory = 'arvind'
    while 1:
        for name in listdir(directory):
            filename = path.join(directory,name)
            image, image_id = load_photo(filename)
            desc = descriptions[image_id]
            in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc, image)
            yield[[in_img,in_seq],out_word]

In [36]:
from os import path
def load_photo(filename):
    image = load_img(filename, target_size=(224,224))
    image = img_to_array(image)
    image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
    image = preprocess_input(image)[0]
    image_id = path.basename(filename).split('.')[0]
    return image,image_id

In [43]:
def create_sequences(tokenizer, max_length, desc, image):
    Ximages, XSeq, y = list(), list(), list()
    vocab_size = len(tokenizer.word_index)+1
    seq = tokenizer.texts_to_sequences([desc])[0]
    for i in range(1,len(seq)):
        in_seq, out_seq = seq[:i], seq[i]
        in_seq = pad_sequences([in_seq],maxlen=max_length)[0]
        out_seq = to_categorical([out_seq],num_classes=vocab_size)[0]
        Ximages.append(image)
        XSeq.append(in_seq)
        y.append(out_seq)
    Ximages, XSeq, y = array(Ximages), array(XSeq), array(y)
    return Ximages, XSeq, y

In [47]:
generator = data_generator(descriptions_1,tokenizer,max_length)

In [48]:
inputs, outputs = next(generator)

In [49]:
print(inputs[0].shape)
print(inputs[1].shape)
print(outputs.shape)

(15, 224, 224, 3)
(15, 16)
(15, 73)
